In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
import time
from WindowGen import WindowGenerator
import NN_net as nn
import utils as utls



In [ ]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
num_file_train = 3
n_input, n_output = 18, 6
titels = ["LPG","LN","HN","LCO","Ffg","Slurry","overall"]

In [ ]:
frames = []
for i in range(num_file_train):
    path =  "./Data/Train_Data_{:n}.xlsx".format(i+1)
    df = pd.read_excel(path)
    frames.append(df)

data = pd.concat(frames)

In [ ]:
data_t = utls.trnsfrm(data)

In [ ]:
ftr_clmns, lbl_clmns = utls.get_key(data.columns.to_list(),n_input,n_output)

In [ ]:
window = WindowGenerator(
    label_width=20,
    input_width=60,
    feature_columns= ftr_clmns+lbl_clmns,
    label_columns= lbl_clmns,
    shift=10,
)

In [ ]:
sz = len(data)
train_df = data_t[0 : int(sz*0.7)]
val_df = data_t[int(sz*0.7) : int(sz*0.9)]
test_df = data_t[int(sz*0.9) :]

In [ ]:
# create datasets from the dataframes
x_train_t, y_train_t = window.create_dataset(train_df)
x_val_t, y_val_t = window.create_dataset(val_df)
x_test_t, y_test_t = window.create_dataset(test_df)

In [ ]:
x_train_t = x_train_t[:,:,:n_input]
x_val_t = x_val_t[:,:,:n_input]
yHis = x_test_t[:,:,n_input:]
x_test_t = x_test_t[:,:,:n_input]
yTrue = utls.invrs_trnsfrm(y_test_t,data,lbl_clmns)


In [ ]:
yHis = utls.invrs_trnsfrm(yHis,data,lbl_clmns)

In [ ]:
test_set_metrics = {}
n_trainable_params = {}
history_set = {}
times_set = {}

In [ ]:
model = nn.dense_model(window=window, units=5)
model_n = "Dense"

start_time = time.time()
history = nn.compile_and_fit(
    model, X=x_train_t, y=y_train_t, val_X=x_val_t, val_y=y_val_t,model_name=model_n)
end_time = (time.time() - start_time)/60


yPred = model.predict(x_test_t)

t = utls.evaluate_forecast(
    y_test_t,
    yPred,
    n_output,
    titels,
    model_n,
    model)

head = ["Output", "mse", "mae", "mape", "r2"]
pd.DataFrame(t).to_excel("./"+model_n+'/err_metrics.xlsx',index=False,header=head)  
test_set_metrics[model_n] = t
history_set[model_n] = history
times_set[model_n] = end_time
n_trainable_params[model_n] = np.sum([np.prod(v.shape) for v in model.trainable_variables])  

In [ ]:
utls.plot_forecast(yHis,yTrue,yPred,window,lbl_clmns,data,model_n)

In [ ]:
utls.plot_rslt_metrics(t,lbl_clmns,model_n)

In [ ]:
utls.plot_rslt_r2_metrics(t,lbl_clmns,model_n)

In [ ]:
model = nn.conv_flex_model(window=window, filters=3,kernel_size=2)
model_n = "CNN"

start_time = time.time()
history = nn.compile_and_fit(
    model, X=x_train_t, y=y_train_t, val_X=x_val_t, val_y=y_val_t,model_name=model_n)
end_time = (time.time() - start_time)/60

yPred = model.predict(x_test_t)

t = utls.evaluate_forecast(
    y_test_t,
    yPred,
    n_output,
    titels,
    model_n,
    model)
head = ["Output", "mse", "mae", "mape", "r2"]
pd.DataFrame(t).to_excel("./"+model_n+'/err_metrics.xlsx',index=False,header=head)  
test_set_metrics[model_n] = t
history_set[model_n] = history
times_set[model_n] = end_time
n_trainable_params[model_n] = np.sum([np.prod(v.shape) for v in model.trainable_variables])  


In [ ]:
utls.plot_forecast(yHis,yTrue,yPred,window,lbl_clmns,data,model_n)

In [ ]:
utls.plot_rslt_metrics(t,lbl_clmns,model_n)

In [ ]:
utls.plot_rslt_r2_metrics(t,lbl_clmns,model_n)

In [ ]:
model = nn.LSTM_model(0,0,27,4.5e-3,True,0.1, window)
model_n = "LSTM"

start_time = time.time()
history = nn.compile_and_fit(
    model, X=x_train_t, y=y_train_t, val_X=x_val_t, val_y=y_val_t,model_name=model_n,lr=1e-4,patience_r=30)
end_time = (time.time() - start_time)/60

yPred = model.predict(x_test_t)

t = utls.evaluate_forecast(
    y_test_t,
    yPred,
    n_output,
    titels,
    model_n,
    model)
head = ["Output", "mse", "mae", "mape", "r2"]
pd.DataFrame(t).to_excel("./"+model_n+'/err_metrics.xlsx',index=False,header=head)  
test_set_metrics[model_n] = t
history_set[model_n] = history
times_set[model_n] = end_time
n_trainable_params[model_n] = np.sum([np.prod(v.shape) for v in model.trainable_variables])  

In [ ]:
utls.plot_forecast(yHis,yTrue,yPred,window,lbl_clmns,data,model_n)

In [ ]:
utls.plot_rslt_metrics(t,lbl_clmns,model_n)

In [ ]:
utls.plot_rslt_r2_metrics(t,lbl_clmns,model_n)

In [ ]:
model = nn.Stacked_LSTM_model(0,0,9,4.5e-3,True,1,window)
model_n = "Stacked-LSTM"

start_time = time.time()
history = nn.compile_and_fit(
    model, X=x_train_t, y=y_train_t, val_X=x_val_t, val_y=y_val_t,model_name=model_n,lr=1e-2)
end_time = (time.time() - start_time)/60
yPred = model.predict(x_test_t)

t = utls.evaluate_forecast(
    y_test_t,
    yPred,
    n_output,
    titels,
    model_n,
    model)
head = ["Output", "mse", "mae", "mape", "r2"]
pd.DataFrame(t).to_excel("./"+model_n+'/err_metrics.xlsx',index=False,header=head)   
test_set_metrics[model_n] = t
history_set[model_n] = history
times_set[model_n] = end_time
n_trainable_params[model_n] = np.sum([np.prod(v.shape) for v in model.trainable_variables])  

In [ ]:
utls.plot_forecast(yHis,yTrue,yPred,window,lbl_clmns,data,model_n)

In [ ]:
utls.plot_rslt_metrics(t,lbl_clmns,model_n)

In [ ]:
utls.plot_rslt_r2_metrics(t,lbl_clmns,model_n)

In [ ]:
model = nn.LSTM_model_encoder_decoder(window=window, encoder_units=27, decoder_units=27)
model_n = "ED-LSTM"

start_time = time.time()
history = nn.compile_and_fit(
    model, X=x_train_t, y=y_train_t, val_X=x_val_t, val_y=y_val_t,model_name=model_n,lr=1e-3)
end_time = (time.time() - start_time)/60

yPred = model.predict(x_test_t)


t = utls.evaluate_forecast(
    y_test_t,
    yPred,
    n_output,
    titels,
    model_n,
    model)
head = ["Output", "mse", "mae", "mape", "r2"]
pd.DataFrame(t).to_excel("./"+model_n+'/err_metrics.xlsx',index=False,header=head)  
test_set_metrics[model_n] = t
history_set[model_n] = history
times_set[model_n] = end_time
n_trainable_params[model_n] = np.sum([np.prod(v.shape) for v in model.trainable_variables])  

In [ ]:
utls.plot_forecast(yHis,yTrue,yPred,window,lbl_clmns,data,model_n)

In [ ]:
utls.plot_rslt_metrics(t,lbl_clmns,model_n)

In [ ]:
utls.plot_rslt_r2_metrics(t,lbl_clmns,model_n)

In [ ]:
model = nn.CNN_LSTM(0,0,0,0,72,21,2,2,6e-5,True,0.25,window)
model_n = "CNN-LSTM"

start_time = time.time()
history = nn.compile_and_fit(
    model, X=x_train_t, y=y_train_t, val_X=x_val_t, val_y=y_val_t,model_name=model_n,lr=1e-4)
end_time = (time.time() - start_time)/60

yPred = model.predict(x_test_t)

t = utls.evaluate_forecast(
    y_test_t,
    yPred,
    n_output,
    titels,
    model_n,
    model)
head = ["Output", "mse", "mae", "mape", "r2"]
pd.DataFrame(t).to_excel("./"+model_n+'/err_metrics.xlsx',index=False,header=head)   
test_set_metrics[model_n] = t
history_set[model_n] = history
times_set[model_n] = end_time
n_trainable_params[model_n] = np.sum([np.prod(v.shape) for v in model.trainable_variables])  

In [ ]:
utls.plot_forecast(yHis,yTrue,yPred,window,lbl_clmns,data,model_n)

In [ ]:
utls.plot_rslt_metrics(t,lbl_clmns,model_n)

In [ ]:
utls.plot_rslt_r2_metrics(t,lbl_clmns,model_n)

In [ ]:
model = nn.BiLSTM_model(0,0,16,4e-3,True,0.15,2,window)
model_n = "Bi-LSTM"

start_time = time.time()
history = nn.compile_and_fit(
    model, X=x_train_t, y=y_train_t, val_X=x_val_t, val_y=y_val_t,model_name=model_n,lr=1e-2)
end_time = (time.time() - start_time)/60

yPred = model.predict(x_test_t)

t = utls.evaluate_forecast(
    y_test_t,
    yPred,
    n_output,
    titels,
    model_n,
    model)
head = ["Output", "mse", "mae", "mape", "r2"]
pd.DataFrame(t).to_excel("./"+model_n+'/err_metrics.xlsx',index=False,header=head)    
test_set_metrics[model_n] = t
history_set[model_n] = history
times_set[model_n] = end_time
n_trainable_params[model_n] = np.sum([np.prod(v.shape) for v in model.trainable_variables])  

In [ ]:
utls.plot_forecast(yHis,yTrue,yPred,window,lbl_clmns,data,model_n)

In [ ]:
utls.plot_rslt_metrics(t,lbl_clmns,model_n)

In [ ]:
utls.plot_rslt_r2_metrics(t,lbl_clmns,model_n)

In [ ]:
reac_train = x_train_t[:,:,[0,1,5,6,9,10,12]]
reg_train = x_train_t[:,:,[2,3,4,8,11,13]]
frac_train = x_train_t[:,:,[7,14,15,16,17]]

reac_val = x_val_t[:,:,[0,1,5,6,9,10,12]]
reg_val = x_val_t[:,:,[2,3,4,8,11,13]]
frac_val = x_val_t[:,:,[7,14,15,16,17]]

reac_test = x_test_t[:,:,[0,1,5,6,9,10,12]]
reg_test = x_test_t[:,:,[2,3,4,8,11,13]]
frac_test = x_test_t[:,:,[7,14,15,16,17]]

n_reac = reac_train.shape[2]
n_reg = reg_train.shape[2]
n_frac = frac_train.shape[2]

In [ ]:
model = nn.MHNN(window=window,n_reac=n_reac,n_reg=n_reg,n_frac=n_frac)
model_n = "MH-LSTM"

start_time = time.time()
history = nn.compile_and_fit(
    model, X={"reac": reac_train, "reg": reg_train, "frac": frac_train},
    y={"fccu": y_train_t},
    val_X={"reac": reac_val, "reg": reg_val, "frac": frac_val},
    val_y={"fccu": y_val_t},
    model_name=model_n,lr=1e-3)
end_time = (time.time() - start_time)/60

yPred = model.predict({"reac": reac_test, "reg": reg_test, "frac": frac_test})

t = utls.evaluate_forecast(
    y_test_t,
    yPred,
    n_output,
    titels,    model_n,
    model)
head = ["Output", "mse", "mae", "mape", "r2"]
pd.DataFrame(t).to_excel("./"+model_n+'/err_metrics.xlsx',index=False,header=head)  
test_set_metrics[model_n] = t
history_set[model_n] = history
times_set[model_n] = end_time
n_trainable_params[model_n] = np.sum([np.prod(v.shape) for v in model.trainable_variables])  

In [ ]:
yPred = model.predict({"reac": reac_test, "reg": reg_test, "frac": frac_test})


In [ ]:
utls.plot_forecast(yHis,yTrue,yPred,window,lbl_clmns,data,model_n)

In [ ]:
yPred = utls.invrs_trnsfrm(yPred, data, lbl_clmns)

In [ ]:
utls.plot_rslts_radar(yTrue,yPred,lbl_clmns,model_n)

In [ ]:
utls.plot_rslt_metrics(t,lbl_clmns,model_n)

In [ ]:
utls.plot_rslt_r2_metrics(t,lbl_clmns,model_n)

In [ ]:
mpl.rcParams['figure.figsize'] = (18, 10)
mpl.rcParams['axes.grid'] = False


fig, ax1 = plt.subplots()
fz = 18
x = np.arange(len(test_set_metrics))
width = 0.4
clr = '#3478b4'
ax1.set_ylabel("MSE", fontsize=fz, color=clr)
ax1.bar(x- width/2 , [test_set_metrics[i][6][1] for i in test_set_metrics.keys()], width, color=clr)
ax1.bar_label(ax1.containers[0],fmt='%.2e', fontsize=12, padding=1)
ax1.set_xticks(ticks=x)
ax1.set_xticklabels(labels=test_set_metrics.keys(), fontsize=fz)
ax1.set_yticklabels(ax1.get_yticks(), fontsize=fz)
ax1.xaxis.set_tick_params(rotation=45)

ax2 = ax1.twinx()

clr = '#f57f16'
ax2.set_ylabel("MAE", fontsize=fz, color=clr)
ax2.bar(x + width/2, [test_set_metrics[i][6][2] for i in test_set_metrics.keys()], width, color=clr)
ax2.bar_label(ax2.containers[0],fmt='%.2e', fontsize=12, padding=1)
ax2.set_yticklabels(ax2.get_yticks(), fontsize=fz)
fig.tight_layout()
plt.savefig("tst_metrics_e" +'.svg', dpi=300)
plt.savefig("tst_metrics_e" +'.eps', dpi=300)

plt.show()

In [ ]:
mpl.rcParams['figure.figsize'] = (18, 10)
mpl.rcParams['axes.grid'] = False



fig, ax3 = plt.subplots()

x = np.arange(len(test_set_metrics))
width = 0.3
values = np.arange(0, 1.2, 0.2)
clr = '#3478b4'
ax3.set_ylabel("R² score", fontsize=fz, color=clr)
ax3.bar(x, [test_set_metrics[i][6][-1] for i in test_set_metrics.keys()], width, color=clr)
ax3.bar_label(ax3.containers[0],fmt='%.4f', fontsize=12)
ax3.set_xticks(ticks=x)
ax3.set_xticklabels(labels=test_set_metrics.keys(), fontsize=fz)
rounded_values = [round(val, 2) for val in values]
ax3.set_yticklabels(rounded_values, fontsize=fz)
ax3.xaxis.set_tick_params(rotation=45)
fig.tight_layout()
plt.savefig("tst_metrics_r2" +'.svg', dpi=300)
plt.savefig("tst_metrics_r2" +'.eps', dpi=300)
plt.show()

In [ ]:
mpl.rcParams['figure.figsize'] = (18, 10)
mpl.rcParams['axes.grid'] = False


fig, ax1 = plt.subplots()

x = np.arange(len(test_set_metrics))
width = 0.3

clr = '#3478b4'
ax1.set_ylabel("Computation time", fontsize=fz, color=clr)
ax1.bar(x, [times_set[i] for i in times_set.keys()], width, color=clr)
ax1.bar_label(ax1.containers[0],fmt='%.3f', fontsize=12)
ax1.set_xticks(ticks=x)
ax1.set_xticklabels(labels=test_set_metrics.keys(), fontsize=fz)
ax1.xaxis.set_tick_params(rotation=45)
ax1.set_yticklabels(ax1.get_yticks(), fontsize=fz)

fig.tight_layout()
plt.savefig("train_time" +'.svg', dpi=300)
plt.savefig("train_time" +'.eps', dpi=300)

plt.show()

In [ ]:
fig, axes = plt.subplots(2, 4, sharex=True, figsize=(10, 5))
axes = axes.ravel()

for i, (key, value) in enumerate(history_set.items()):

    axes[i].plot(history_set[key].history["loss"], label="train")
    axes[i].plot(history_set[key].history["val_loss"], label="validation")
    axes[i].set(title=(key,n_trainable_params[key]))
    
    axes[i].legend()

plt.tight_layout()
plt.savefig("train_history" +'.svg', dpi=fig.dpi)
plt.savefig("train_history" +'.eps', dpi=fig.dpi)
